In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_mistralai import ChatMistralAI
from langchain_community.tools import tool
import json
from dotenv import load_dotenv
load_dotenv()
from langchain_core.messages import HumanMessage
import asyncio
from pprint import pprint

llm = ChatMistralAI(model_name="mistral-large-latest")

tools = []

# data = None
# with open('actions.json', 'r') as file:
#     data = json.load(file)


# def parse_id(id_string):
#     return id_string.split('_', 1)[1] if '_' in id_string else id_string

# for action in data['actions']:
#     exec(action['code'])
#     tools.append(globals()[parse_id(action['title'])])

@tool
def get_geocoding(city) -> list:
    """
    Retrieve latitude and longitude of a city via API.

    Args:
        city (str): City name.

    Returns:
        list: A list of geocoding data containing 'name', 'latitude', 'longitude', and 'country'.
              Example:
              [
                  {
                      "name": "City Name",
                      "latitude": 12.34,
                      "longitude": 56.78,
                      "country": "Country Name"
                  }
              ]
              
              Prints error message and status code if the request fails.
    """

    import requests
    api_url = 'https://api.api-ninjas.com/v1/geocoding?city={}'.format(city)
    response = requests.get(api_url, headers={'X-Api-Key': "meaUO1Ucu4ZTlk7qtQMibA==cpyHoWVnOLLm6QW2"})
    if response.status_code == requests.codes.ok:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)

tools = [get_geocoding]


agent = create_react_agent(llm, tools)


def process_stream(stream):
    data = []
    try:
        for chunk in stream:
            print("Received chunk:", end=" ")
            if 'agent' in chunk:
                data.append(chunk['agent']['messages'][0].json())
            if 'tools' in chunk:
                data.append(chunk['tools']['messages'][0].json())
            pprint(chunk)

            # if hasattr(chunk, 'content'):
            #     print(chunk.content, end='', flush=True)
            # elif isinstance(chunk, dict):
            #     pprint(chunk)
            # else:
            #     print(f"Unexpected chunk type: {type(chunk)}")
    except Exception as e:
        print(f"An error occurred: {e}")


    print("\n\n\n------------------------------------------------------")

    pprint(data)

async def main():
    stream = agent.stream({"messages": [HumanMessage(content="What is the geocoding of indore")]})
    # Use run_in_executor to run the synchronous function in a separate thread
    await asyncio.get_event_loop().run_in_executor(None, process_stream, stream)




if __name__=="__main__":
    asyncio.run(main())

In [1]:
from langgraph.prebuilt import create_react_agent
from langchain_mistralai import ChatMistralAI
from langchain_community.tools import tool
import json
from dotenv import load_dotenv
load_dotenv()
from langchain_core.messages import HumanMessage
import asyncio
from pprint import pprint


llm = ChatMistralAI(model_name="mistral-large-latest")

tools = []

# data = None
# with open('actions.json', 'r') as file:
#     data = json.load(file)


# def parse_id(id_string):
#     return id_string.split('_', 1)[1] if '_' in id_string else id_string

# for action in data['actions']:
#     exec(action['code'])
#     tools.append(globals()[parse_id(action['title'])])

@tool
def get_geocoding(city) -> list:
    """
    Retrieve latitude and longitude of a city via API.

    Args:
        city (str): City name.

    Returns:
        list: A list of geocoding data containing 'name', 'latitude', 'longitude', and 'country'.
              Example:
              [
                  {
                      "name": "City Name",
                      "latitude": 12.34,
                      "longitude": 56.78,
                      "country": "Country Name"
                  }
              ]
              
              Prints error message and status code if the request fails.
    """

    import requests
    api_url = 'https://api.api-ninjas.com/v1/geocoding?city={}'.format(city)
    response = requests.get(api_url, headers={'X-Api-Key': "meaUO1Ucu4ZTlk7qtQMibA==cpyHoWVnOLLm6QW2"})
    if response.status_code == requests.codes.ok:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)


In [2]:
tools.append(get_geocoding)

llm_with_tools = llm.bind_tools(tools=tools)

In [4]:
respone = llm_with_tools.invoke("What is the geocoding of indore")

In [6]:
tool_call = respone.tool_calls[0]

In [7]:
tool_call['args']

{'city': 'indore'}

In [8]:
get_geocoding.invoke(tool_call['args'])

[{'name': 'Indore',
  'latitude': 22.7203616,
  'longitude': 75.8681996,
  'country': 'IN',
  'state': 'Madhya Pradesh'},
 {'name': 'Simrol',
  'latitude': 22.539445,
  'longitude': 75.9115643,
  'country': 'IN',
  'state': 'Madhya Pradesh'},
 {'name': 'Kampel',
  'latitude': 22.6189602,
  'longitude': 76.0498576,
  'country': 'IN',
  'state': 'Madhya Pradesh'},
 {'name': 'Indore',
  'latitude': 24.9772666,
  'longitude': 77.858262,
  'country': 'IN',
  'state': 'Madhya Pradesh'},
 {'name': 'Ralamandal Village',
  'latitude': 22.64809095,
  'longitude': 75.90138634026997,
  'country': 'IN',
  'state': 'Madhya Pradesh'}]

In [9]:
agent = create_react_agent(llm, tools)